## PROJECT

### This is a project to scrap data from the web and store the results in both a text file as well as the SQLite database.


#### The CNN Money’s Market Movers website (https://money.cnn.com/data/hotstocks/ ) tracks the most active stocks on a real time basis. You will first write Python scripts that collect the list of Most Actives tickers only from the above website. Next, your programs should take these ticker symbols and build a comma separated text file (called stocks.txt) with data about each stock from the website: https://finance.yahoo.com/quote/AMD?p=AMD&.tsrc=fin-srch-v1 which gives the quote for ticker symbol AMD as an example. The data to be collected from the Yahoo Finance site should include (Actual text as seen in the website is in brackets):
    OPEN price (Open)
    AVERAGE VOLUME (Avg. Volume)
    PE RATIO (PE Ratio (TTM))

#### In addition to the stocks.txt file, the data should also be stored in an SQLite database called StocksDatabase in the directory that your Jupyter Notebook code will be executed from. The StocksDatabase should have a table called StocksTable that contains the following columns and types:
    Ticker TEXT
    OpenPrice REAL
    AvgVolume INTEGER
    PERatio REAL


<font color = red>**Algorithm for the program :**</font>

<font color = blue>Step - 1</font> Import from the url library the url handling modules also import re module and import sqlite3 module respectively <br>
<font color = blue>Step - 2</font> Initiate try block and handle exceptions incase of web page is disruptive and create else statement for displaying the web scrapping program execution and completion message  <br>
<font color = blue>Step - 3</font> Make a request to a web page using get method to send a get request to the specified url and then convert the html content to a text file <br>
<font color = blue>Step - 4</font> In the text file using regex command re.findall and re.DOTALL pattern match with multiple lines span the Most active tickers and from those tickers further pattern match the ticker symbols <br>
<font color = blue>Step - 5</font> Now in the new web page url with format() and a for loop iterate through the list of most active ticker symbols and send a get request to the specified active ticker url and then convert the html content to a text file  <br>
<font color = blue>Step - 6</font> Create empty lists each for storing data to be used in textfile, database , also for each individual ticker url links and specific data to be collected from each of the most active tickers <br>
<font color = blue>Step - 7</font> Inside the for loop using regex commands re.findall and re.DOTALL pattern match from each of the most active ticker web pages data and their respective Open Price , Average Volume and PE ratio and convert them to float, integer and using replace (), then append them into the empty list created for storing data specific to each most active ticker <br>
### NOTE : We are assuming PE ratio with N/A values or no earnings as 0.0 Real Number for our program construct convenience
<font color = blue>Step - 8</font> Data collected as list elements from specific most active tickers is then appened in other empty list created each for textfile and database constructs<br>
<font color = blue>Step - 9</font> Create a CSV text file with textfile list elements by using join and map() and adding new line character at the end ,then open and create a text file named 'stocks.txt' and using writelines method write all list elements from the textfile list into the text file named 'stocks.txt' and then close the file <br>
<font color = blue>Step - 10</font> Finally, with connection established with the SQLite database and the cursor we create a 'StocksDatabase.sqlite' database file and using SQL commands we Drop and create a Table named StocksTable and insert each Most active ticker's in TEXT and their specific stock values of OpenPrice in REAL, Average Volume in INTEGER and PE Ratio in REAL types respectively, using (execute, executemany for multiple inserts) and then we commit those records in the database and close both connections with the cursor and the SQLite database , this concludes the program  <br>

### NOTE : In this program the execution takes a while , so please wait until program completion message is displayed.




In [ ]:
from urllib.request import urlopen                      # importing from the url library url handling modules
import requests                                         # importing the requests module to send HTTP requests 
import re                                               # importing the re module to search a string for a match
import sqlite3                                          # importing the sqlite3 module to perform SQL commands

try:                                                    # beginning of try block              
    hotstocks_url = 'https://money.cnn.com/data/hotstocks/' # web page url
    hotstocks_fhand = requests.get(hotstocks_url)       # makes a request to a web page using get request to the specified url
    hotstocks_text = hotstocks_fhand.text               # returns html content into a text file
    
    most_active = re.findall('<h3>Most Actives</h3>.*?</table>',hotstocks_text,re.DOTALL) # using regex commands do pattern matching for string
    ticker_symbols = re.findall('class="wsod_symbol">([^ <]*)',most_active[0])            # using regex commands pattern match most active ticker symbol
    
    ticker_url = 'https://finance.yahoo.com/quote/{urlticker}?p={urlticker}&.tsrc=fin-srch-v1' # web page url
    print('Program is scraping data from the web......please wait!')# print statement to display beginning of the execution of the web scraping program
    
    textfile_list = []                                  # empty list for storing data to be appended into textfile
    database_list = []                                  # empty list for storing data to be appended into database list
   
    for ticker in ticker_symbols:                       # for loop to iterate through each most active tickers list
        mystocks_fhand = requests.get(ticker_url.format(urlticker = ticker)) # makes a get request for each ticker web page
        mystocks_text = mystocks_fhand.text             # returns html content into a text file      
        
        ticker_link = []                                # empty list for storing html content from the web page 
        ticker_link = re.findall('OPEN-value.*?EPS',mystocks_text,re.DOTALL)# using regex commands pattern match and store the html content
        stockdata = []                                  # empty list for storing data specific to each most active ticker   
        stockdata.append(ticker)                        # appends each ticker into the empty list created as 0th element

        data = re.findall('OPEN-value.*?([^ >]*)</span></td></tr>',ticker_link[0],re.DOTALL) # using regex commands pattern match openprice of the ticker
        stockdata.append(float(data[0]))                #  convert numeric string character to float and append into stockdata list

        data = re.findall('AVERAGE_VOLUME_3MONTH-value.*?([^ >]*)</span></td></tr>',ticker_link[0],re.DOTALL)# using regex commands pattern match avgvolume of the ticker
        stockdata.append(int(data[0].replace(',','')))  # using replace() remove comma in string, convert numeric string character to integer and append into stockdata list
        
        data = re.findall('PE_RATIO-value.*?([^ >]*)</span></td></tr>',ticker_link[0],re.DOTALL)# using regex commands pattern match PEratio of the ticker
        stockdata.append(float(data[0].replace('N/A','0.0')))# using replace () remove N/A in string, convert to assumed float value, append into stockdata list

        database_list.append(stockdata)                      # appends respective stockdata lists in database list
        textfile_list.append(((',').join(map(str,stockdata)))+'\n')# creates a CSV file with list elements join using map() and add newline character at the end of each line
    
        fhand = open('stocks.txt','w')                  # opens stocks file in write mode
        fhand.writelines(textfile_list)                 # writes all list elements into the text file   
        fhand.close()                                   # closes the file 
       
        conn = sqlite3.connect('StocksDatabase.sqlite') # connects or creates StocksDatabase in sqlite database
        cur = conn.cursor()                             # initialize cursor and get cursor handle  

        cur.execute('DROP TABLE IF EXISTS StocksTable') # drop StocksTable if it exists
        cur.execute('CREATE TABLE StocksTable (Ticker TEXT, OpenPrice REAL, AvgVolume INTEGER, PERatio REAL)') # creates StocksTable with column names and types 
        cur.executemany('INSERT INTO StocksTable VALUES (?,?,?,?);',database_list)# insert all records/rows at once from database list
        conn.commit()                                   # commit/write the record inserts into database
        
        cur.close()                                     # closes the current cursor object
        conn.close()                                    # closes the connection to current database  
    
except:                                                 # raise an exception to catch URL error
    print('Website is currently down. Please try again!')      
else:                                                   # if try block doesn't raise an error this displays the program execution statement when it completes
    print('The stocks.txt CSV File is created')
    print('The StocksDatabase is created: Open in SQLite Browser')